# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

2016-03-26 11:25:48,569 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: /tmp/graphlab_server_1459005946.log


This non-commercial license of GraphLab Create is assigned to mrp412@nyu.edu and will expire on October 28, 2016. For commercial licensing options, visit https://dato.com/buy/.


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].a/|'/stype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 2.023658     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 2.092364     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [21]:
model_all['coefficients'].print_rows(num_rows=18)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 
### Answer: bathrooms, sqft_living, sqft_living_sqrt, grade, sqft_above

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [22]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [23]:
import numpy as np
l1_penalty_values = np.logspace(1, 7, num=13)
min_error = None
best_l1_penalty = None
validation_errors = []
for l1_penalty in l1_penalty_values:
    print l1_penalty
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=None,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    price_predicted = model.predict(validation)
    RSS = ((price_predicted-validation['price'])**2).sum()
    validation_errors.append(RSS)
    if min_error is None or RSS < min_error:
        min_error = RSS
        best_l1_penalty = l1_penalty

10.0
31.6227766017
100.0
316.227766017
1000.0
3162.27766017
10000.0
31622.7766017
100000.0
316227.766017
1000000.0
3162277.66017
10000000.0


In [24]:
print validation_errors
print "Best l1_penalty: " + str(best_l1_penalty)
print "Best RSS: " + str('{0:f}'.format(min_error))

[625766285142459.9, 625766285362394.1, 625766286057885.0, 625766288257224.6, 625766295212186.8, 625766317206080.5, 625766386760658.1, 625766606749278.5, 625767302791634.1, 625769507643886.2, 625776517727024.0, 625799062845467.0, 625883719085425.2]
Best l1_penalty: 10.0
Best RSS: 625766285142459.875000


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [25]:

model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=None,
                                              l2_penalty=0., l1_penalty=best_l1_penalty)
price_predicted = model.predict(testing)
RSS = ((price_predicted-testing['price'])**2).sum()
print RSS

1.56983602382e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [26]:
model.get("coefficients").print_rows(num_rows=18)
print "# Non zeros : " + str(model['coefficients']['value'].nnz())

+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4272128   |  None  |
|     bedrooms     |  None |  7936.96767903   |  None  |
| bedrooms_square  |  None |  936.993368193   |  None  |
|    bathrooms     |  None |  25409.5889341   |  None  |
|   sqft_living    |  None |  39.1151363797   |  None  |
| sqft_living_sqrt |  None |  1124.65021281   |  None  |
|     sqft_lot     |  None | 0.00348361822299 |  None  |
|  sqft_lot_sqrt   |  None |  148.258391011   |  None  |
|      floors      |  None |   21204.335467   |  None  |
|  floors_square   |  None |  12915.5243361   |  None  |
|    waterfront    |  None |  601905.594545   |  None  |
|       view       |  None |  93312.8573119   |  None  |
|    condition     |  None |  6609.03571245   |  None  |
|      grade       |  None |  6206.93999188   |  None  |
|    sqft_above    |  None |  4

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [27]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [28]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [29]:
min_error = None
best_l1_penalty = None
validation_errors = []
for l1_penalty in l1_penalty_values:
    print l1_penalty
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=None,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    print "# Non zeros : " + str(model['coefficients']['value'].nnz())
    
    price_predicted = model.predict(validation)
    RSS = ((price_predicted-validation['price'])**2).sum()
    validation_errors.append(RSS)
    if min_error is None or RSS < min_error:
        min_error = RSS
        best_l1_penalty = l1_penalty

print "Best l1_penalty: " + str(best_l1_penalty)
print "Best RSS: " + str('{0:f}'.format(min_error))

100000000.0
# Non zeros : 18
127427498.57
# Non zeros : 18
162377673.919
# Non zeros : 18
206913808.111
# Non zeros : 18
263665089.873
# Non zeros : 17
335981828.628
# Non zeros : 17
428133239.872
# Non zeros : 17
545559478.117
# Non zeros : 17
695192796.178
# Non zeros : 17
885866790.41
# Non zeros : 16
1128837891.68
# Non zeros : 15
1438449888.29
# Non zeros : 15
1832980710.83
# Non zeros : 13
2335721469.09
# Non zeros : 12
2976351441.63
# Non zeros : 10
3792690190.73
# Non zeros : 6
4832930238.57
# Non zeros : 5
6158482110.66
# Non zeros : 3
7847599703.51
# Non zeros : 1
10000000000.0
# Non zeros : 1
Best l1_penalty: 100000000.0
Best RSS: 627492659875075.500000


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [30]:
l1_penalty_min = 2976351441.63
l1_penalty_max = 3792690190.73

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [31]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [32]:
min_error = None
best_l1_penalty = None
validation_errors = []
for l1_penalty in l1_penalty_values:
    print l1_penalty
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=None,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    print "# Non zeros : " + str(model['coefficients']['value'].nnz())
    
    price_predicted = model.predict(validation)
    RSS = ((price_predicted-validation['price'])**2).sum()
    validation_errors.append(RSS)
    if min_error is None or RSS < min_error:
        min_error = RSS
        best_l1_penalty = l1_penalty

print "Best l1_penalty: " + str(best_l1_penalty)
print "Best RSS: " + str('{0:f}'.format(min_error))

2976351441.63
# Non zeros : 10
3019316638.95
# Non zeros : 10
3062281836.27
# Non zeros : 10
3105247033.59
# Non zeros : 10
3148212230.91
# Non zeros : 10
3191177428.24
# Non zeros : 10
3234142625.56
# Non zeros : 10
3277107822.88
# Non zeros : 10
3320073020.2
# Non zeros : 8
3363038217.52
# Non zeros : 8
3406003414.84
# Non zeros : 8
3448968612.16
# Non zeros : 7
3491933809.48
# Non zeros : 7
3534899006.8
# Non zeros : 7
3577864204.12
# Non zeros : 7
3620829401.45
# Non zeros : 6
3663794598.77
# Non zeros : 6
3706759796.09
# Non zeros : 6
3749724993.41
# Non zeros : 6
3792690190.73
# Non zeros : 6
Best l1_penalty: 2976351441.63
Best RSS: 966925692362084.500000


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [33]:

model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=None,
                                              l2_penalty=0., l1_penalty=3448968612.16)
price_predicted = model.predict(validation)
RSS = ((price_predicted-validation['price'])**2).sum()
print RSS
print model.get("coefficients").print_rows(num_rows=18)

1.04693748875e+15
+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.192544 |  None  |
|     bedrooms     |  None | 661.722717782 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15873.9572593 |  None  |
|   sqft_living    |  None | 32.4102214513 |  None  |
| sqft_living_sqrt |  None | 690.114773313 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2899.42026975 |  None  |
|    sqft_above    |  None | 30.0115753022 |  None  |
|  sqft_ba